## 相当于是一个阅读笔记吧，会将[wtfpython: A collection of surprising Python snippets and lesser-known features](https://github.com/satwikkansal/wtfpython) (中文版: [中文](https://github.com/leisurelicht/wtfpython-cn))中遇到一些我认为值得思考的代码片段记下来。

## Section: Strain your brain!

### >  Return return everywhere!

In [1]:
def some_func():
    try:
        return 'from_try'
    finally:
        return 'from_finally'

In [2]:
some_func()

'from_finally'

<hr>
💡
* <font color='red'>"try...finally" 语句的 try 中执行 return, break 或 continue 后, finally 子句依然会执行.</font>
* 函数的返回值由最后执行的 return 语句决定. 所以 finally 子句中的 return 将是最后执行的语句.

也就是说，尽量避免在try语句中写return语句(毕竟try 语句仅用来处理有可能出错的代码).

另外，一些语言(如C#)会禁止你在finally 语句中 return.

### > For what?

In [3]:
some_string = "wtf"
some_dict = {}
for i, some_dict[i] in enumerate(some_string):
    pass

In [4]:
some_dict

{0: 'w', 1: 't', 2: 'f'}

<hr>
💡
Python 语法 中对 for 的定义是:`for_stmt: 'for' exprlist 'in' testlist ':' suite ['else' ':' suite]`

其中 exprlist 指分配目标. 这意味着对可迭代对象中的每一项都会执行类似 {exprlist} = {next_value} 的操作. 在上例中，每次循环都有some_dict[i] = enumerate(some_string)产生的字符串, 即 some_dict[0] = 'w', ...... 

一个例子：该循环只循环一次？

In [5]:
for i in range(4):
    print(i)
    i = 10

0
1
2
3


### > Evaluation time discrepancy/执行时机差异

In [6]:
array = [1, 8, 15]
g = (x for x in array if array.count(x) == 1)
array = [2, 8, 22]

list(g)

[8]

In [7]:
array_1 = [1,2,3,4]
g1 = (x for x in array_1)
array_1 = [1,2,3,4,5]

array_2 = [1,2,3,4]
g2 = (x for x in array_2)
array_2[:] = [1,2,3,4,5]

print(list(g1))
print(list(g2))

[1, 2, 3, 4]
[1, 2, 3, 4, 5]


💡 

* 在生成器表达式中, in 子句在声明时执行, 而条件子句则是在运行时执行.
* 所以在运行前, array 已经被重新赋值为 [2, 8, 22], 因此对于之前的 1, 8 和 15, 只有 count(8) 的结果是等于 1 的, 所以生成器只会生成 8. <font color='red'>???</font>
* 第二部分中 g1 和 g2 的输出差异则是由于变量 array_1 和 array_2 被重新赋值的方式导致的.
* 在第一种情况下, array_1 被绑定到新对象 [1,2,3,4,5], 因为 in 子句是在声明时被执行的， 所以它仍然引用旧对象 [1,2,3,4](并没有被销毁).
* 在第二种情况下, 对 array_2 的切片赋值将相同的旧对象 [1,2,3,4] 原地更新为 [1,2,3,4,5]. 因此 g2 和 array_2 仍然引用同一个对象(这个对象现在已经更新为 [1,2,3,4,5]).

### > What's wrong with booleans?

In [8]:
# 一个简单的例子, 统计下面可迭代对象中的布尔型值的个数和整型值的个数
mixed_list = [False, 1.0, "some_string", 3, True, [], False]
integers_found_so_far = 0
booleans_found_so_far = 0

for item in mixed_list:
    if isinstance(item, int):
        integers_found_so_far += 1
    elif isinstance(item, bool):
        booleans_found_so_far += 1

In [9]:
integers_found_so_far, booleans_found_so_far

(4, 0)

In [10]:
another_dict = {}
another_dict[True] = "JavaScript"
another_dict[1] = "Ruby"
another_dict[1.0] = "Python"

In [11]:
another_dict[True]

'Python'

In [12]:
>>> some_bool = True
>>> "wtf"*some_bool
'wtf'
>>> some_bool = False
>>> "wtf"*some_bool
''

''

💡
* 布尔值是 int 的子类
* 所以 True 的整数值是 1, 而 False 的整数值是 0.